# Morsy 6/29/20
## Speckle Image Process

### 1. Image Read

In [1]:
def spkim(path, image_num):
    # spkim(path, image_num) This function takes image path and speckle number as an input argouments and plots the image 
    #on pixel-sized axes. Also, the function will pair each "spk_num" with the corresponding image in global dictionary "im_dic"
    # Note: the "path" must be typed in qoutation marks, and "image_num" must be an intger or a float. The function uses
    #image number to lable each speckle image correctly in case of the function is used to process multiple images.
    #
    
    from PIL import Image 
    import matplotlib.pyplot as plt
    global im_dic

    im_dic1 = {}
    y = Image.open(path , "r" ) 
    im_dic1 = {('spk_'+ str(image_num)): y}# This line creates a global dictionary that pairs the "spk_num" to imported image
    im_dic.update(im_dic1)
    
    %matplotlib inline
    dx_m = 1*10**-6  #pixel size in meter
    Dx_m = 1e-3      #image full size
    extent = (-Dx_m/2 * 1e6, +Dx_m/2 * 1e6, -Dx_m/2 * 1e6, +Dx_m/2 * 1e6) 
    plt.imshow(y, extent = extent, cmap = 'inferno')
    plt.xlabel('Horizontal Position [um]')
    plt.ylabel('Vertical Positon [um]')
    plt.title('Speckel ' + str(image_num))
    plt.colorbar()
    plt.show()

In [2]:
#spkim('C:\\Users\\User\\Downloads\\speckle_one.png', 1)
#spkim("C:\\Users\\User\\Downloads\\speckle_two.png", 2)
#print(im_dic)
#print(list(im_dic.keys())[list(im_dic.values()).index(im_dic['spk_1'])])


### 2. Frequency Scale

In [3]:
def setfreq():
    # setfreq() is a function theat does not accept any inputs, rather it sets the frequency scale of the fourier
    #transformed image allowing the user to make comparasions between
    #the frequencies of fourier transformed images 
    
    import numpy as np
    dx_m = 1*10**-6  #pixel size in meter
    Dx_m = 1e-3      #image full size
    x_m = np.linspace(-Dx_m/2, Dx_m/2, Dx_m/dx_m + 1)
    real_axis = x_m
    fs = 1/(real_axis[1]-real_axis[0])
    Nfft=len(real_axis)
            
    df = fs/Nfft
    f_cpm = np.linspace(0,(fs-df),Nfft) - (fs-np.mod(Nfft,2)*df)/2
    
    #print(dx_m)
    #print(Dx_m)
    #print(x_m)
    #print(real_axis)
    #print(Nfft)

In [4]:
#setfreq()

### 3. Fourier Transform

In [5]:
def ftim(spk_num):
    # ftim(spk_num) is a function that: First, it converts imput image into a nympy array. Second, it preforms
    # a fourier transformation. Lastly, it plots the transformation on frequncy scale.
    # Note: Input of this function should be in the form of an image file that has been already imported. If "spkim()" function
    # was used prior to "ftim()", then input can be optained using the keys of the global dictionary im_dic. 
    #EX: ftim(im_dic['spk_1'])
    
    import numpy as np
    import matplotlib.pyplot as plt
    
    dx_m = 1*10**-6  #pixel size in meter
    Dx_m = 1e-3      #image full size
    x_m = np.linspace(-Dx_m/2, Dx_m/2, Dx_m/dx_m + 1)
    real_axis = x_m
    fs = 1/(real_axis[1]-real_axis[0])
    Nfft=len(real_axis)
            
    df = fs/Nfft
    f_cpm = np.linspace(0,(fs-df),Nfft) - (fs-np.mod(Nfft,2)*df)/2
    
    %matplotlib inline
    data_1 = np.asarray(spk_num) # Converts the image into a numpy array
    Y1 = np.fft.fftshift(np.fft.fft2(data_1))  # it rearranges the fourier transform to make it symetrical about zero

    df_cpm = f_cpm[1] - f_cpm[0]  #pixel size in meter
    Df_cpm = f_cpm[-1] - f_cpm[0]      #image full size
    extent = (-Df_cpm/2 * 1e-6, +Df_cpm/2 * 1e-6, -Df_cpm/2 * 1e-6, +Df_cpm/2 * 1e-6) 

    plt.imshow(np.abs(Y1)**0.1, extent = extent)
    plt.xlabel('Freq[/um]')
    plt.ylabel('Freq[/um]')
    plt.title('PSD '+ list(im_dic.keys())[list(im_dic.values()).index(spk_num)] + ' Magnitude') # adds the dictionary(spk_num)
                                                                                                    #key to plot title
    plt.show()

In [6]:
#ftim(im_dic['spk_1'])
#ftim(im_dic['spk_2'])

### 4. Circle Sum

In [7]:
def circsum(spk_num):
    
    import numpy as np
    import matplotlib.pyplot as plt
    
    dx_m = 1*10**-6  #pixel size in meter
    Dx_m = 1e-3      #image full size
    x_m = np.linspace(-Dx_m/2, Dx_m/2, Dx_m/dx_m + 1)
    real_axis = x_m
    fs = 1/(real_axis[1]-real_axis[0])
    Nfft=len(real_axis)
    
    df = fs/Nfft
    f_cpm = np.linspace(0,(fs-df),Nfft) - (fs-np.mod(Nfft,2)*df)/2
    df_cpm = f_cpm[1] - f_cpm[0]  #pixel size in meter
    Df_cpm = f_cpm[-1] - f_cpm[0]      #image full size
            
    
    
    (Fx, Fy) = np.meshgrid(f_cpm, f_cpm)
    Z = Fx**2 + Fy**2    # Equation of a circule
    r = df_cpm * 100      # Radious of the ring
    dr = df_cpm * 1     # Thikness of the ring
#a = np.where(Z >= 10)
#a = np.where(np.abs(Z - 10) < 1) 
    a = np.where(np.abs(Z - r**2) < dr**2)   # Creates an array that contained zeros everywhere exept for the radii
    A = Z * 0
    A[a] = 1

    extent = (-Df_cpm/2 * 1e-6, +Df_cpm/2 * 1e-6, -Df_cpm/2 * 1e-6, +Df_cpm/2 * 1e-6)
    
    N = 501
    circ_sum_1 = np.zeros(N)
    for i in np.arange(N):
        r = df_cpm * i      # Radious of the ring
        dr = df_cpm     # Thikness of the ring
    #a = np.where(np.abs(Z - r**2) <= dr**2)   # Creates an array that contained zeros everywhere exept for the radii
        a = np.where((Z >= r**2) & (Z < (r + dr)**2))
        A = Z * 0
        A[a] = 1
        
        data_1 = np.asarray(spk_num) # Converts the image into a numpy array
        Y1 = np.fft.fftshift(np.fft.fft2(data_1))  # it rearranges the fourier transform to make it symetrical about zero

        circ_sum_1[i] = np.sum(np.abs(Y1) * A) / np.sum(A)
        temp_1 = np.sum(np.abs(Y1) *A)
    plt.plot(f_cpm[500:] * 1e-6, circ_sum_1)
    plt.ylim((0, 10000))
    plt.title(list(im_dic.keys())[list(im_dic.values()).index(spk_num)])
    plt.xlabel('Freq[/um]')
    plt.ylabel('Circular Sum of PSD')
    plt.show()


In [8]:
#circsum(im_dic['spk_1'])
#circsum(im_dic['spk_2'])